In [5]:
import random as rnd
import pandas as pd
from IPython.display import display
import sys
import math
import copy
import keyboard
from IPython.display import clear_output

def sign(x):
    if x >= 0:
        return 1
    elif x < 0:
        return -1


###############################################################################################
########   KEY   ##############################################################################
###############################################################################################
class lcl_key:
    
    def __init__(self, iv_x, iv_y, iv_color):
        self.x = iv_x
        self.y = iv_y
        self.color = iv_color
    
    def print(self):
        print('KEY: x:', self.x, ' y:',  self.y, ' color:', self.color) 
    
###############################################################################################
########   DOOR   #############################################################################
###############################################################################################
class lcl_door:
    
    def __init__(self, iv_x, iv_y, iv_color, iv_walkable):
        self.x = iv_x
        self.y = iv_y
        self.color = iv_color
        self.opened = False
        self.walkable = iv_walkable
      
    
    def open(self, io_key):
        if io_key.color == self.color:
            self.opened = True
            return True
        else:
            return False
    
    
    def is_opened(self):
        return self.opened
    
###############################################################################################
########   CORRIDOR   #########################################################################
###############################################################################################
class lcl_corridor:
    
    def __init__(self, io_start_room, io_end_room, start_x, start_y, end_x, end_y):
        self.start   = io_start_room
        self.end     = io_end_room
        self.start_x = start_x
        self.start_y = start_y
        self.end_x   = end_x
        self.end_y   = end_y
        
    
    def reverse(self):
        self.start_x, self.end_x = self.end_x, self.start_x
        self.start_y, self.end_y = self.end_y, self.start_y
        self.end,     self.start = self.start, self.end
    
    
    def print(self):
        print('CORRIDOR: (', str(self.start_x), ', ', str(self.start_y), ') ---> (', str(self.end_x), ', ', str(self.end_y), ')', sep='')
        
###############################################################################################
########   ROOM   #############################################################################
###############################################################################################

class lcl_room:
    
    def __init__(self, iv_quater, iv_x_width, iv_y_width, iv_room_percent_limit, iv_min_room_width):
        la_data        = self.generate_room(iv_quater, iv_x_width, iv_y_width, iv_room_percent_limit, iv_min_room_width)
        self.x         = la_data['x']
        self.y         = la_data['y']
        self.dx        = la_data['dx']
        self.dy        = la_data['dy']
        if self.dx < 0 or self.dy < 0:
            raise KeyError
        self.square    = abs((abs(self.dx)-1)*(abs(self.dy)-1))
        self.quater    = iv_quater
        self.x_max     = self.x + self.dx
        self.y_max     = self.y + self.dy
        self.doors     = 0
        self.corridors = { 'left':  0,
                           'right': 0
                         }
        self.corners   = { 'x1y1': { 
                               'x': self.x + 1,
                               'y': self.y + 1 
                           },
                           'x1y2': { 
                               'x': self.x + 1,
                               'y': self.y_max - 1
                           },
                           'x2y1': { 
                               'x': self.x_max - 1,
                               'y': self.y + 1
                           },
                           'x2y2': {
                               'x': self.x_max - 1,
                               'y': self.y_max - 1
                           }
                         }

    
    def find_closest_corners(self, io_room):
        min_dist = sys.maxsize
        self_corner = 0
        other_corner = 0
        for key_i, value_i in self.corners.items():
            for key_j, value_j in io_room.corners.items():
                curr_dist = math.sqrt( ( value_j['x'] - value_i['x'])**2 \
                                + ( value_j['y'] - value_i['y'])**2 )
                if curr_dist < min_dist:
                    min_dist = curr_dist
                    self_corner = key_i
                    other_corner = key_j
        return { 'start': self_corner, 
                 'end'  : other_corner }
        
        
    def get_coord_limit_from_quater(self, iv_quater, iv_x_width, iv_y_width, iv_room_percent_limit):
        if iv_quater == 0:
            return {'x_min': 0, \
                    'y_min': 0, \
                    'x_max': int((iv_x_width // 2) * iv_room_percent_limit), \
                    'y_max': int((iv_y_width // 2) * iv_room_percent_limit) }
        elif iv_quater == 3:
            return {'x_min': int((iv_x_width // 2 ) * iv_room_percent_limit) , \
                    'y_min': 0, \
                    'x_max': int((iv_x_width - 1) * iv_room_percent_limit), \
                    'y_max': int((iv_y_width // 2) * iv_room_percent_limit)}
        elif iv_quater == 1:
            return {'x_min': 0 , \
                    'y_min': int((iv_y_width // 2 ) * iv_room_percent_limit), \
                    'x_max': int((iv_x_width // 2) * iv_room_percent_limit), \
                    'y_max': int((iv_y_width-1) * iv_room_percent_limit)}
        elif iv_quater == 2:
            return {'x_min': int((iv_x_width // 2 ) * iv_room_percent_limit), \
                    'y_min': int((iv_y_width // 2 ) * iv_room_percent_limit), \
                    'x_max': int((iv_x_width-1) * iv_room_percent_limit), \
                    'y_max': int((iv_y_width-1) * iv_room_percent_limit) }

        
    def generate_room(self, iv_quater, iv_x_width, iv_y_width, iv_room_percent_limit, iv_min_room_width):
        la_coord_lim = self.get_coord_limit_from_quater(iv_quater, \
                                                        iv_x_width, \
                                                        iv_y_width, \
                                                        iv_room_percent_limit)
        lv_x = rnd.randint(int(la_coord_lim['x_min']), \
                           int(la_coord_lim['x_max'] - iv_min_room_width))
        lv_y = rnd.randint(int(la_coord_lim['y_min']), \
                           int(la_coord_lim['y_max'] - iv_min_room_width))
        lv_x_max = rnd.randint(lv_x + iv_min_room_width, \
                               int(la_coord_lim['x_max']))
        lv_y_max = rnd.randint(lv_y + iv_min_room_width, \
                               int(la_coord_lim['y_max']))
    
        lv_dx = lv_x_max - lv_x
        lv_dy = lv_y_max - lv_y
        
        ev_return = {'x': lv_x, \
                    'dx': lv_dx,\
                     'y': lv_y, \
                    'dy': lv_dy }
        return ev_return
        
         
    def is_initial(self):
        if self.x  == 0 and \
           self.y  == 0 and \
           self.dx == 0 and \
           self.dy == 0:
            return True
        else:
            return False
    
    
    def is_inside(self, iv_x, iv_y):
        if self.x < iv_x < self.x_max and \
           self.y < iv_y < self.y_max:
            return True
        else:
            return False
    
    
    def is_on_border(self, iv_x, iv_y):
        if ( self.x == iv_x or self.x_max == iv_x ) and self.y < iv_y < self.y_max or \
           ( self.y == iv_y or self.y_max == iv_y ) and self.x < iv_x < self.x_max:
            return True
        else:
            return False       
    
    
    def get_left_quater(self):
        return (self.quater + 1) % 4
    
    
    def get_right_quater(self):
        return (self.quater - 1) % 4
    
    
    def get_diag_quater(self):
        return (self.quater + 2) % 4
    
    
    def swap_corridors(self):
        self.corridors['left'], self.corridors['right'] = self.corridors['right'], self.corridors['left']
    
    
    def print(self):
        print('ROOM -->')
        print('quater: ' + str(self.quater) + ' doors: ' + str(self.doors) + ' square: ' + str(self.square) \
              + ' (' + str(self.x) + ',' + str(self.y) + ') ---> ' \
              '(' + str(self.x + self.dx) + ',' + str(self.y + self.dy) + ')' )
        print('CORNERS: ', self.corners)
        print('LEFT', end=' ')
        if isinstance(self.corridors['left'], (int, float, complex)):
            print(self.corridors['left'])
        else:
            self.corridors['left'].print()
        print('RIGHT', end=' ')
        if isinstance(self.corridors['right'], (int, float, complex)):
            print(self.corridors['right'])
        else:
            self.corridors['right'].print()
        
###############################################################################################
########   MAZE   #############################################################################
###############################################################################################
        
class lcl_maze_generator:
  

    def __init__(self, iv_x_width, iv_y_length, iv_mv_room_num, iv_room_percent_limit, iv_min_room_square, iv_min_room_width):
        self.mv_x_width = iv_x_width # ОСЬ x
        self.mv_y_width = iv_y_length # ОСЬ y
        self.mv_room_num = iv_mv_room_num
        self.min_room_square = iv_min_room_square
        self.mv_room_percent_limit = iv_room_percent_limit
        self.mv_min_room_width = iv_min_room_width
        self.init_maze()
        self.ml_rooms = []
        self.ml_doors = []
        self.ml_keys = []

        
    def init_all(self):
        self.__init__(self.mv_x_width, self.mv_y_width, self.mv_room_num, self.mv_room_percent_limit, self.min_room_square, self.mv_min_room_width)
    
    
    def init_maze(self):
        self.ml_maze = self.get_empty_maze(self.mv_x_width, self.mv_y_width)
    
    
    def init_rooms(self):
        self.ml_rooms = []
    
    
    def remove_key(self, io_key):
        self.ml_keys.remove(io_key)
    
    
    def remove_door(self, io_door):
        self.ml_doors.remove(io_door)
    
    
    def get_rooms(self):
        return self.ml_rooms   
            
    
    def find_room_by_quater(self, iv_quater):
        for room in self.ml_rooms:
            if room.quater == iv_quater:
                return room
                break
        return -1
    
    
    def get_maze(self):
        return self.ml_maze

    
    def print_rooms(self):
        for room in self.ml_rooms:
            room.print()
    
    
    def print_maze_simple(self):
        for i in self.ml_maze:
            print(i)
    
    
    def print_maze(self):
        # Устанавливаем параметры отображения
        pd.set_option('display.max_rows', 10)  # Максимальное количество отображаемых строк
        pd.set_option('display.max_columns', 5)  # Максимальное количество отображаемых столбцов
        # Стилизуем DataFrame
        display(pd.DataFrame(self.ml_maze) \
                  .replace(0, '') \
                  .style \
                  .set_properties(**{'text-align': 'center'}))  # Центрирование текста
    
    
    def get_empty_maze(self, iv_x_width, iv_y_width):
        return [ [0]*iv_y_width for i in range(iv_x_width) ]
    
    
    def find_rooms_by_coord(self, iv_x, iv_y):
        # count = 0
        result = []
        for room in self.ml_rooms:
            if room.is_on_border(iv_x, iv_y):
                 result.append(room)
        return result
    
    
    def visualize_maze(self):
        # Устанавливаем параметры отображения
        lo_temp_maze = copy.deepcopy(self)
        lo_temp_maze.visualize_objects()
        
        pd.set_option('display.max_rows', 10)  # Максимальное количество отображаемых строк
        pd.set_option('display.max_columns', 5)  # Максимальное количество отображаемых столбцов
        # Стилизуем DataFrame
        display(pd.DataFrame(lo_temp_maze.ml_maze) \
                  .replace(0, '') \
                  .style \
                  .set_properties(**{'text-align': 'center'}))  # Центрирование текста
    
    
    def get_empty_maze(self, iv_x_width, iv_y_width):
        return [ [0]*iv_y_width for i in range(iv_x_width) ]
    
    
    def find_rooms_by_coord(self, iv_x, iv_y):
        result = []
        for room in self.ml_rooms:
            if room.is_on_border(iv_x, iv_y):
                 result.append(room)
        return result   
    
    
    def generate_room_walls(self, ia_room):
        lv_x  = ia_room.x
        lv_y  = ia_room.y
        lv_dx = ia_room.dx
        lv_dy = ia_room.dy
        
        try:
            for i in range(lv_x, lv_x+lv_dx+1):
                self.ml_maze[i][lv_y] = 1
                self.ml_maze[i][lv_y+lv_dy] = 1

            for i in range(lv_y, lv_y+lv_dy+1):
                self.ml_maze[lv_x][i] = 1
                self.ml_maze[lv_x+lv_dx][i] = 1
        except Exception as e:
            print('i',i,'x',lv_x,'y',lv_y,'dx',lv_dx,'dy',lv_dy)
            print('len_x', len(self.ml_maze))
            raise Exception
    
    
    def generate_all_walls(self):
        for i in range(0, len(self.ml_maze)):
            for j in range(0, len(self.ml_maze[i])):
                # Если мы внутри комнаты
                if self.is_inside_any_room(i, j) == True:
                    if self.ml_maze[i][j] == 2:
                        self.ml_maze[i][j] = 0
                else:
                    if self.ml_maze[i][j] == 2:
                        self.ml_maze[i][j] = 0
                    elif self.ml_maze[i][j] == 0:
                        self.ml_maze[i][j] = 1
    
    
    def generate_all_room_walls(self):
        for lv_room in self.ml_rooms:
            if not lv_room.is_initial( ):
                self.generate_room_walls(lv_room)
                

    def check_intersection(self, room_a, room_b):
        # true - есть пересечение; false - нет пересечения
        # Если хотя бы одна вершина одного находится в области другого, то пересечение
        # так как прямоугольники не вращаются, то когда вершина первого находится в области другого
        # то и вершина другого находится в области первого
        # в связи с чем достаточно проверить наличие вершин комнаты А в комнате Б
        return not ( room_a.y > room_b.y_max or room_a.y_max < room_b.y or room_a.x_max < room_b.x or room_a.x > room_b.x_max )
        
        
    def is_inside_any_room(self, iv_x, iv_y):
        # rv_result = False
        for room in self.ml_rooms:
            if room.is_inside(iv_x, iv_y) == True:
                return True
        return False
    
    
    def is_on_border_any_room(self, iv_x, iv_y):
        for room in self.ml_rooms:
            if room.is_on_border(iv_x, iv_y) == True:
                return True
        return False
    
    
    def check_available_coord(self,  iv_x, iv_y, iv_no_player):
        for door in self.ml_doors:
            if door.x == iv_x and door.y == iv_y:
                return False
    
        for key in self.ml_keys:
            if key.x == iv_x and key.y == iv_y:
                return False
        
        if iv_no_player == False:
            # Игрок не сможет спавниться на ключе или на финальной двери
            if self.player_x == iv_x and self.player_y == iv_y:
                return False
        
        if self.is_inside_any_room(iv_x, iv_y) or \
           self.is_on_border_any_room(iv_x, iv_y):
            if self.ml_maze[iv_x][iv_y] != 0 and \
               self.ml_maze[iv_x][iv_y] != 2:
                return False
        else:
            return False
        
        return True
    
    
    def is_good_for_door(self, iv_x, iv_y):
        if self.check_available_coord(iv_x, iv_y) and \
           self.is_on_border_any_room(iv_x, iv_y):
            return True
        else:
            return False
    
    
    def generate_rooms(self):
        self.ml_rooms = []
        # Каждая из 4 комнат будет находиться в центре своей четверти
        # Выбираем первую четверь и генерируем там комнату
        self.ml_rooms.append(lcl_room(0, self.mv_x_width, self.mv_y_width, self.mv_room_percent_limit, self.mv_min_room_width))
        # test = 0
        for lv_index in range(1, self.mv_room_num):
            lo_new_room = lcl_room(lv_index, self.mv_x_width, self.mv_y_width, self.mv_room_percent_limit, self.mv_min_room_width)
            
            for lv_old_index in range(0, lv_index):
                lo_old_room = self.ml_rooms[lv_old_index]
        
                lv_stop = 0
                while lo_new_room.is_initial() == True or \
                      self.check_intersection(lo_old_room, lo_new_room) == True or \
                      lo_new_room.square < self.min_room_square:
                    lv_stop += 1
                    if lv_stop >= 10000:
                        print('ENDLESS LOOP')
                        break
                    lo_new_room = lcl_room(lv_index, self.mv_x_width, \
                                                     self.mv_y_width, \
                                                     self.mv_room_percent_limit, \
                                                     self.mv_min_room_width )
            self.ml_rooms.append(lo_new_room)
            
        self.ml_rooms.sort(key = lambda obj: (obj.quater))

    
    def generate_corridors_4rooms(self):
        # Строим коридоры между комнатами по часовой стрелке по четвертям
        ld_used = {}
        
        for i in range(0, self.mv_room_num):
            lo_start = self.ml_rooms[i]
            if i == len(self.ml_rooms) - 1:
                lo_end = self.ml_rooms[0]
            else:
                lo_end = self.ml_rooms[i + 1]
            
            ld_corners = lo_start.find_closest_corners(lo_end)
            lv_x_start = lo_start.corners[ld_corners['start']]['x']
            lv_y_start = lo_start.corners[ld_corners['start']]['y']
            lv_x_end   = lo_end.corners[ld_corners['end']]['x']
            lv_y_end   = lo_end.corners[ld_corners['end']]['y']
            lv_dx = lv_x_end - lv_x_start
            lv_dy = lv_y_end - lv_y_start
            
            lv_half_x = int(lv_x_start + lv_dx // 2)
            
            lv_door_x_start = 0
            lv_door_y_start = 0
            lv_door_x_end = 0
            lv_door_y_end = 0
            # try:
            y = 'NO'
            x = 'NO'
            for x in range(lv_x_start, lv_x_end + sign(lv_dx), sign(lv_dx)):    

                # print('x', x)
                if ( x < lv_half_x and lv_dx > 0 ) or \
                   ( x > lv_half_x and lv_dx < 0 ):
                    if not self.is_inside_any_room(x, lv_y_start):
                        self.ml_maze[x][lv_y_start] = 2

                    if lo_start.is_on_border(x, lv_y_start) == True:
                        lv_door_x_start = x
                        lv_door_y_start = lv_y_start


                    if lo_end.is_on_border(x, lv_y_start) == True:
                        lv_door_x_end = x
                        lv_door_y_end = lv_y_start

                    if lo_end.is_inside(x, lv_y_start) == True:
                        break

                elif x == lv_half_x:
                    for y in range(lv_y_start, lv_y_end + sign(lv_dy), sign(lv_dy)):
                        if not self.is_inside_any_room(x, y):
                            self.ml_maze[x][y] = 2

                        if lo_start.is_on_border(x, y) == True:
                            lv_door_x_start = x
                            lv_door_y_start = y


                        if lo_end.is_on_border(x, y) == True:
                            lv_door_x_end = x
                            lv_door_y_end = y

                        if lo_end.is_inside(x, y) == True:
                            break
                                 
                    if lo_end.is_inside(x, y) == True:
                        break
                else:
                    if not self.is_inside_any_room(x, lv_y_end):
                        self.ml_maze[x][lv_y_end] = 2

                if lo_start.is_on_border(x, lv_y_end) == True:
                    lv_door_x_start = x
                    lv_door_y_start = lv_y_end


                if lo_end.is_on_border(x, lv_y_end) == True:
                    lv_door_x_end = x
                    lv_door_y_end = lv_y_end

                    if lo_end.is_inside(x, lv_y_end) == True:
                        break
                        
            lo_corridor = lcl_corridor(lo_start, \
                                       lo_end, \
                                       lv_door_x_start, \
                                       lv_door_y_start, \
                                       lv_door_x_end, \
                                       lv_door_y_end)
            # Переворачиваем туннель
            lo_reversed = copy.deepcopy(lo_corridor)
            lo_reversed.reverse()
            
            

            if lo_start.is_on_border(lo_corridor.start_x,lo_corridor.start_y) == True:
                lo_start.corridors['left'] = lo_corridor
                lo_end.corridors['right'] = lo_reversed
            elif lo_end.is_on_border(lo_corridor.start_x,lo_corridor.start_y) == True:
                lo_start.corridors['left'] = lo_reversed
                lo_end.corridors['right'] = lo_corridor
        
            # Считаем количество дверей в комнатах
            if lv_door_x_start in ld_used.keys() and \
               lv_door_y_start in ld_used[lv_door_x_start].keys():
                lv_used = True
            else:
                lv_used = False
                ld_used[lv_door_x_start] = {}
                ld_used[lv_door_x_start][lv_door_y_start] = 1

            if lv_used == False:
                lv_rooms = self.find_rooms_by_coord(lv_door_x_start,lv_door_y_start)

                for room in lv_rooms:
                    room.doors += 1

            
            if lv_door_x_end in ld_used.keys() and \
               lv_door_y_end in ld_used[lv_door_x_end].keys():
                lv_used = True
            else:
                lv_used = False
                ld_used[lv_door_x_end] = {}
                ld_used[lv_door_x_end][lv_door_y_end] = 1
            
            if lv_used == False:
                lv_rooms = self.find_rooms_by_coord(lv_door_x_end,lv_door_y_end)

                for room in lv_rooms:
                    room.doors += 1
        
      
    
    def generate_doors_keys_spawn(self):
        # Выбираем случайную комнату, в которой будем генерировать финальную дверь
        ll_free_colors = ['b','g','o']

        lo_start_room = self.ml_rooms[rnd.randint(0, self.mv_room_num - 1)]

        # В моей схеме для простоты всегда будем делать конфигурацию для левой стороны
        # Но, на самом деле за левой стороной может скрываться как левая, так и правая

        lo_real_left_room  = self.find_room_by_quater(lo_start_room.get_left_quater())
        lo_real_right_room = self.find_room_by_quater(lo_start_room.get_right_quater())

        lo_pseudo_left_room = rnd.choice([lo_real_left_room, lo_real_right_room])

        if lo_pseudo_left_room == lo_real_left_room:
            lo_pseudo_start_room = copy.deepcopy(lo_start_room)
            lo_pseudo_right_room = lo_real_right_room
        else:
            lo_pseudo_start_room = copy.deepcopy(lo_start_room)
            lo_pseudo_start_room.swap_corridors()
            lo_pseudo_right_room = lo_real_left_room

        lo_diag_room = self.find_room_by_quater(lo_pseudo_start_room.get_diag_quater())


        # Генерируем координаты точки спавна игрока
        self.player_x = rnd.randint(lo_pseudo_start_room.x + 1, lo_pseudo_start_room.x_max - 1)
        self.player_y = rnd.randint(lo_pseudo_start_room.y + 1, lo_pseudo_start_room.y_max - 1)

        while self.check_available_coord(self.player_x, self.player_y, True) == False:
            self.player_x = rnd.randint(lo_pseudo_start_room.x + 1, lo_pseudo_start_room.x_max - 1)
            self.player_x = rnd.randint(lo_pseudo_start_room.y + 1, lo_pseudo_start_room.y_max - 1)



        # Генерируем координаты и цвет "синего" ключа
        lv_blue_color = rnd.choice(ll_free_colors)
        ll_free_colors.remove(lv_blue_color)        
        lo_pseudo_blue_key = lcl_key(rnd.randint(lo_pseudo_start_room.x + 1, lo_pseudo_start_room.x_max - 1), \
                                     rnd.randint(lo_pseudo_start_room.y + 1, lo_pseudo_start_room.y_max - 1), \
                                     lv_blue_color)


        while self.check_available_coord(lo_pseudo_blue_key.x, lo_pseudo_blue_key.y, False) == False:
            lo_pseudo_blue_key.x = rnd.randint(lo_pseudo_start_room.x + 1, lo_pseudo_start_room.x_max - 1)
            lo_pseudo_blue_key.y = rnd.randint(lo_pseudo_start_room.y + 1, lo_pseudo_start_room.y_max - 1)

        self.ml_keys.append(lo_pseudo_blue_key)

        # Генерируем "синюю" дверь в конце правого коридора
        lo_blue_door = lcl_door( lo_pseudo_start_room.corridors['right'].end_x, \
                                 lo_pseudo_start_room.corridors['right'].end_y, \
                                 lv_blue_color, \
                                 False )

        self.ml_doors.append(lo_blue_door)



        # Генерируем координаты и цвет "зелёного" ключа
        lv_green_color = rnd.choice(ll_free_colors)
        ll_free_colors.remove(lv_green_color)        
        lo_pseudo_green_key = lcl_key(rnd.randint(lo_pseudo_right_room.x + 1, lo_pseudo_right_room.x_max - 1), \
                                      rnd.randint(lo_pseudo_right_room.y + 1, lo_pseudo_right_room.y_max - 1), \
                                      lv_green_color)


        while self.check_available_coord(lo_pseudo_green_key.x, lo_pseudo_green_key.y, False) == False:
            lo_pseudo_green_key.x = rnd.randint(lo_pseudo_right_room.x + 1, lo_pseudo_right_room.x_max - 1)
            lo_pseudo_green_key.y = rnd.randint(lo_pseudo_right_room.y + 1, lo_pseudo_right_room.y_max - 1)

        self.ml_keys.append(lo_pseudo_green_key)

        # Генерируем "зелёную" дверь в конце левого коридора
        lo_green_door = lcl_door( lo_pseudo_start_room.corridors['left'].end_x, \
                                  lo_pseudo_start_room.corridors['left'].end_y, \
                                  lv_green_color, \
                                  False)

        self.ml_doors.append(lo_green_door)



        # Генерируем координаты и цвет "оранжевого" ключа в псевдо-левой комнате
        lv_orange_color = rnd.choice(ll_free_colors)
        ll_free_colors.remove(lv_orange_color)        
        lo_pseudo_orange_key = lcl_key(rnd.randint(lo_pseudo_left_room.x + 1, lo_pseudo_left_room.x_max - 1), \
                                       rnd.randint(lo_pseudo_left_room.y + 1, lo_pseudo_left_room.y_max - 1), \
                                       lv_orange_color)


        while self.check_available_coord(lo_pseudo_orange_key.x, lo_pseudo_orange_key.y, False) == False:
            lo_pseudo_orange_key.x = rnd.randint(lo_pseudo_left_room.x + 1, lo_pseudo_left_room.x_max - 1)
            lo_pseudo_orange_key.y = rnd.randint(lo_pseudo_left_room.y + 1, lo_pseudo_left_room.y_max - 1)

        self.ml_keys.append(lo_pseudo_orange_key)

        # Генерируем "оранжевую" дверь
        if lo_diag_room.doors == 1:
            lo_orange_door = lcl_door( lo_diag_room.corridors['left'].start_x, \
                                       lo_diag_room.corridors['left'].start_y, \
                                       lv_orange_color, \
                                       False)
            self.ml_doors.append(lo_orange_door)
        else:
            lo_orange_door = lcl_door( lo_diag_room.corridors['left'].start_x, \
                                       lo_diag_room.corridors['left'].start_y, \
                                       lv_orange_color, \
                                       False )
            self.ml_doors.append(lo_orange_door)
            lo_orange_door = lcl_door( lo_diag_room.corridors['right'].start_x, \
                                       lo_diag_room.corridors['right'].start_y, \
                                       lv_orange_color, \
                                       False)
            self.ml_doors.append(lo_orange_door)





        # Генерируем координаты и цвет "финального" ключа в псевдо-левой комнате     
        lo_final_key = lcl_key(rnd.randint(lo_diag_room.x + 1, lo_diag_room.x_max - 1), \
                               rnd.randint(lo_diag_room.y + 1, lo_diag_room.y_max - 1), \
                               'f')


        while self.check_available_coord(lo_final_key.x, lo_final_key.y, False) == False:
            lo_final_key.x = rnd.randint(lo_diag_room.x + 1, lo_diag_room.x_max - 1)
            lo_final_key.y = rnd.randint(lo_diag_room.y + 1, lo_diag_room.y_max - 1)

        self.ml_keys.append(lo_final_key)


        # Генерируем координаты финальной двери
        lo_final_door = lcl_door(rnd.randint(lo_pseudo_start_room.x + 1, lo_pseudo_start_room.x_max - 1), \
                                 rnd.randint(lo_pseudo_start_room.y + 1, lo_pseudo_start_room.y_max - 1), \
                                 'f', \
                                  True )

        while self.ml_maze[lo_final_door.x][lo_final_door.y] == 2:
            lo_final_door.x = rnd.randint(lo_pseudo_start_room.x + 1, lo_pseudo_start_room.x_max - 1)
            lo_final_door.y = rnd.randint(lo_pseudo_start_room.y + 1, lo_pseudo_start_room.y_max - 1)

        self.ml_doors.append(lo_final_door)
        
        
    
    def visualize_objects(self):
        
        self.ml_maze[self.player_x][self.player_y] = 'PL'
        
        for door in self.ml_doors:
            if door.opened == False:
                if door.x == self.player_x and door.y == self.player_y:
                    pl = 'PL'
                else:
                    pl = ''
                self.ml_maze[door.x][door.y] = pl + 'd' + door.color
        
        for key in self.ml_keys:
            if key.x == self.player_x and key.y == self.player_y:
                pl = 'PL'
            else:
                pl = ''
            self.ml_maze[key.x][key.y] = pl + 'k' + key.color
                
    
    def regenerate_maze(self):
        self.init_all()
        self.generate_rooms()
        self.generate_corridors_4rooms()
        self.generate_doors_keys_spawn()
        self.generate_all_walls()
        
        
    def find_objects(self, iv_x, iv_y):
        result = []
        # Если существует точка в пределах лабиринта
        if len(self.ml_maze) > iv_x and \
           len(self.ml_maze[iv_x]) > iv_y:
            
            # Если это не стена
            if self.ml_maze[iv_x][iv_y] != 1:
                for door in self.ml_doors:
                    if door.x == iv_x and door.y == iv_y:
                        result.append(door)

                for key in self.ml_keys:
                    if key.x == iv_x and key.y == iv_y:
                        result.append(key)
            else:
                result = []
                result.append('wall')
        else:
            result = []
            result.append('space')

        if result == []:
            result.append('floor')
        
        return result
    
    
    def what_is_up(self, iv_x, iv_y):
        return self.find_objects(iv_x - 1, iv_y)
    
    
    def what_is_left(self, iv_x, iv_y):
        return self.find_objects(iv_x, iv_y - 1)
    
    
    def what_is_right(self, iv_x, iv_y):
        return self.find_objects(iv_x, iv_y + 1)
    
    
    def what_is_down(self, iv_x, iv_y):
        return self.find_objects(iv_x + 1, iv_y)
    
###############################################################################################
########   PLAYER   ###########################################################################
###############################################################################################
class lcl_player:
    
    def __init__(self, io_maze, iv_x, iv_y):
        self.mo_maze = io_maze
        self.x = iv_x
        self.y = iv_y
        self.inventory = []
        self.chosen_item = -1
        self.chosen_item_index = -1

    def choose_item(self, iv_index):
        if len(self.inventory) > iv_index:
            self.chosen_item = self.inventory[iv_index]
            self.chosen_item_index = iv_index
            return self.chosen_item
        else:
            return False
            
            
    def inventory_to_str(self):
        result = []
        result.append('Инвентарь:')
        if self.inventory == []:
            result[0] += ' пусто'
        else:
            for i in range(len(self.inventory)):
                item = self.inventory[i]
                temp = str(i+1) + ' Тип: '
                if 'lcl_key' in str(type(item)):
                    temp += '<Ключ> Значение: ' + str(item.color)
                result.append(temp)
        result.append('Выбрано:' + str(self.chosen_item_index + 1))
        return result
            
    
    def print_inventory(self):
        for i in self.inventory_to_str():
            print(i)
    
    
    def take_object(self):
        ld_objects = self.mo_maze.find_objects(self.x, self.y)
        if str(type(ld_objects[0])) != "<class 'str'>":
            for item in ld_objects:
                if 'lcl_key' in str(type(item)):
                    self.inventory.append(item)
                    self.mo_maze.remove_key(item)
                
    
    def try_open_door(self, io_door):
        needed_key_color = io_door.color
        self.chosen_item
        if 'lcl_key' in str(type(self.chosen_item)) and \
           self.chosen_item.color == io_door.color:
            door_answer = io_door.open(self.chosen_item)
            if door_answer == True:
                self.mo_maze.remove_door(io_door)
#                 self.inventory.remove(self.chosen_item)
                return door_answer
        else:
            return False
                
    
    def move_up(self):
        self.x -= 1
        self.take_object()
        self.update_maze()
                
    
    def move_left(self):
        self.y -= 1
        self.take_object()
        self.update_maze()
                
    
    def move_right(self):
        self.y += 1
        self.take_object()
        self.update_maze()
                
    
    def move_down(self):
        self.x += 1
        self.take_object()
        self.update_maze()
    
    
    def update_maze(self):
        self.mo_maze.player_x = self.x
        self.mo_maze.player_y = self.y
    
    
    def call_what_is(self, iv_direction):
        if iv_direction == 'up':
            return self.mo_maze.what_is_up(self.x, self.y)
        elif iv_direction == 'left':
            return self.mo_maze.what_is_left(self.x, self.y)
        elif iv_direction == 'right':
            return self.mo_maze.what_is_right(self.x, self.y)
        elif iv_direction == 'down':
            return self.mo_maze.what_is_down(self.x, self.y)
    
    
    def call_move(self, iv_direction):
        if iv_direction == 'up':
            return self.move_up()
        elif iv_direction == 'left':
            return self.move_left()
        elif iv_direction == 'right':
            return self.move_right()
        elif iv_direction == 'down':
            return self.move_down()
    
    
    def try_move(self, iv_direction):
        ld_objects = self.call_what_is(iv_direction)
        # Проверяем, что мы не в космосе
        lv_avaliable = False
        result_obj = []
        if ld_objects[0] != 'space' and ld_objects[0] != 'wall' and ld_objects[0] != 'floor':
            for obj in ld_objects:
                if 'lcl_key' in str(type(obj)):
                    self.call_move(iv_direction)
                    lv_avaliable = True
                    result_obj.append(obj)
                elif 'lcl_door' in str(type(obj)):
                    if self.try_open_door(obj) == True:
                        self.call_move(iv_direction)
                        lv_avaliable = True
                        result_obj.append(obj)
                    elif obj.walkable == True:
                        self.call_move(iv_direction)
                        lv_avaliable = True
                        result_obj.append(obj)
                    else:
                        lv_avaliable = False
                        result_obj.append(obj)
        elif ld_objects[0] == 'floor':
            self.call_move(iv_direction)
            return { 'sucsess': True, 'obj': ld_objects[0] } 
        else:
            return { 'sucsess': False, 'obj': ld_objects[0] }
        return  { 'sucsess': lv_avaliable, 'obj': result_obj }
        
    
    def print_move_return(self, iv_return):
        if iv_return['sucsess'] == False:
            print('Передвижение неуспешно. Причина:', iv_return['obj'])
            if 'list' in str(type(iv_return['obj'])):
                for obj in iv_return['obj']:
                    if 'lcl_door' in str(type(obj)):
                        print('Дверь заперта!')
        else:                
            print('Передвижение успешно. События:')
            for obj in iv_return['obj']:
                if 'lcl_key' in str(type(obj)):
                    print('Ключ найден!')
                if 'lcl_door' in str(type(obj)):
                    print('Дверь открыта!')
    
    
    def print_choice_return(self, iv_return):
        print('Был выбран предмет: ',end='')
        if 'lcl_key' in str(type(iv_return)):
            print(' Тип: <Ключ> Значение:')
            if 'lcl_key' in str(type(iv_return)):
                print('Ключ Значение: ' + str(iv_return.color))
    
###############################################################################################
########   GAME   #############################################################################
###############################################################################################

class lcl_game:
    
    def __init__(self):
        self.MC_ROOM_NUM = 4
        self.MC_ROOM_PERCENT_SIZE = 1
        self.MC_MIN_ROOM_SQUARE = 5 
        self.MC_MIN_ROOM_WIDTH = 3 # LC_MIN_ROOM_WIDTH * 2 + 2 < LC_MIN_FIELD_WIDTH
        self.MC_MIN_FIELD_WIDTH = self.MC_MIN_ROOM_WIDTH * 2 + 3
        self.MC_MAX_FIELD_WIDTH = self.MC_MIN_FIELD_WIDTH + 5
        self.regenerate_maze_and_size()
    
    def regenerate_maze_and_size(self):
        self.mo_maze = lcl_maze_generator(rnd.randint(self.MC_MIN_FIELD_WIDTH, self.MC_MAX_FIELD_WIDTH), \
                                          rnd.randint(self.MC_MIN_FIELD_WIDTH, self.MC_MAX_FIELD_WIDTH),  \
                                          self.MC_ROOM_NUM, \
                                          self.MC_ROOM_PERCENT_SIZE, \
                                          self.MC_MIN_ROOM_SQUARE, \
                                          self.MC_MIN_ROOM_WIDTH)
        self.mo_maze.regenerate_maze()
        self.mo_player = lcl_player(self.mo_maze, self.mo_maze.player_x, self.mo_maze.player_y)
        
        
    def regenerate_maze(self):
        self.mo_maze.regenerate_maze()
        self.mo_player = lcl_player(self.mo_maze, self.mo_maze.player_x, self.mo_maze.player_y)
        
    def play(self):
        while True:
            print('Текущие координаты: ', self.mo_player.x,self.mo_player.y)
            self.mo_player.print_inventory()
                    
            self.mo_maze.visualize_maze()
            try:
                # Wait for the next event.
                event = keyboard.read_event()
                if event.event_type == keyboard.KEY_UP:
                    clear_output(wait=True)
                    print(event.name)
                    if event.name == 'esc':
                        print('END GAME')
                        break
                        
                    elif event.name == 'up':
                        lv_player_return = self.mo_player.try_move('up')
                        self.mo_player.print_move_return(lv_player_return)
                    elif event.name == 'left':
                        lv_player_return = self.mo_player.try_move('left')
                        self.mo_player.print_move_return(lv_player_return)
                    elif event.name == 'right':
                        lv_player_return = self.mo_player.try_move('right')
                        self.mo_player.print_move_return(lv_player_return)
                    elif event.name == 'down':
                        lv_player_return = self.mo_player.try_move('down')
                        self.mo_player.print_move_return(lv_player_return)
                    elif event.name == '1':
                        lv_player_return = self.mo_player.choose_item(0)
                        self.mo_player.print_choice_return(lv_player_return)
                    elif event.name == '2':
                        lv_player_return = self.mo_player.choose_item(1)
                        self.mo_player.print_choice_return(lv_player_return)
                    elif event.name == '3':
                        lv_player_return = self.mo_player.choose_item(2)
                        self.mo_player.print_choice_return(lv_player_return)
                    elif event.name == '4':
                        lv_player_return = self.mo_player.choose_item(3)
                        self.mo_player.print_choice_return(lv_player_return)
                        
                    else:
                        continue
            except KeyboardInterrupt:
                print('END GAME')
                break
        
        

In [ ]:
lo_game = lcl_game()
lo_game.play()

up
Передвижение неуспешно. Причина: [<__main__.lcl_door object at 0x000002BBE07B1050>]
Дверь заперта!
Текущие координаты:  6 3
Инвентарь:
1 Тип: <Ключ> Значение: g
Выбрано:0


,0,1,2,3,4,5,6,7,8,9
0,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,,,1
2,1,1,1,1,1,1,1,,kf,1
3,1,1,,,,,do,,,1
4,1,1,,kb,1,1,1,do,1,1
5,1,1,1,dg,1,1,1,,1,1
6,1,1,1,PL,1,1,1,,,1
7,1,,,,1,1,1,,,1
8,1,,,,1,1,1,,ko,1
9,1,,df,,,,db,,,1


# 